In [ ]:
import os
import pickle
import numpy as np
import tensorflow as tf

In [ ]:
from tensorflow.contrib.layers import fully_connected
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from Scripts.CreateTrainingBatches import CreateTrainingBatches

In [ ]:
with open(os.path.join('Data','data_X_y.p'), 'rb') as handle:
    data_X_y = pickle.load(handle)

with open(os.path.join('Data','training_params.p'), 'rb') as handle:
    training_params = pickle.load(handle)
    
rev_vocab_dict = training_params['rev_vocab_dict']

In [ ]:
X_train, X_valid, X_test = [data_X_y['X_train'],data_X_y['X_valid'], data_X_y['X_test']]
y_train, y_valid, y_test = [data_X_y['y_train'],data_X_y['y_valid'], data_X_y['y_test']]

In [ ]:
def return_actual_text(x, rev_vocab_dict):
    actual_text = " ".join([rev_vocab_dict[word_id] for word_id in x  if rev_vocab_dict[word_id]!='my_dummy'])
    return actual_text

In [ ]:
doc_num = 1
return_actual_text(X_train[doc_num], rev_vocab_dict), y_train[doc_num]


In [ ]:
vocab_size = training_params['vocab_size']
mlb = MultiLabelBinarizer()
X_train = mlb.fit_transform(X_train)
X_valid = mlb.transform(X_valid)

In [ ]:
create_training_batches_object = CreateTrainingBatches(X_train, y_train, X_valid, y_valid)

In [ ]:
def print_metrics(np_prob, np_y):
    neg_accuracy = np.mean((np_prob<0.5)[(np_y==0)])
    pos_accuracy = np.mean((np_prob>0.5)[(np_y==1)])
    accuracy = np.mean((pos_accuracy, neg_accuracy))
    print('Negative accuracy',neg_accuracy)
    print('Positive accuracy',pos_accuracy)
    print('Accuracy', accuracy)
    return accuracy

Given an input $x$, the model should be able to map it to $\hat{y}$ $$
\begin{align}
\hat{y}  = \sigma(w^Tx+b) \
\end{align}$$ The paramters of the model are the weights $w$ and the bias $b$

In [ ]:
tf.reset_default_graph()
learning_rate = 0.01

X = tf.placeholder(dtype=tf.float32,shape=[None, vocab_size], name='X')
weights = tf.Variable(tf.random_uniform([X.get_shape().as_list()[1], 1], minval=-1.0, maxval=1.0), name='weights')
bias = tf.zeros(1, name='bias')
logits = tf.matmul(X, weights) + bias
prob = tf.nn.sigmoid(logits, name='prob')

The loss is an estimate the difference between the true label $y$ and the predicted label $\hat{y}$ $$\begin{align}
\ L(\hat{y},y) = - [y log(\hat{y}) + (1 - y) log(1 - \hat{y})] \
\end{align}$$

In [ ]:
y = tf.placeholder(tf.float32, [None, 1], name='y')
losses = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits, name='x_entropy')
loss = tf.reduce_mean(losses, name='loss')

Update the Parameter to minimze the loss Here $\alpha$ is the learning rate and $\frac{\partial L}{\partial w}$ is the partial derivative of Loss $L$ with respect to weight  $w$ $$\begin{align}
\ w = w - \alpha \frac{\partial L}{\partial w}\\
\ b = b - \alpha \frac{\partial L}{\partial b}\\
\end{align}$$

In [ ]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss, name='train_op')

In [ ]:
file_writer = tf.summary.FileWriter('Data/tf_logs/logistic_regression', tf.get_default_graph())

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.InteractiveSession()
init.run()

In [ ]:
for i in range(500):
    x_train_samples, y_train_samples = create_training_batches_object.create_training_data()
    
    _, np_prob, np_y, np_loss = sess.run([training_op, prob, y, loss], 
                                         feed_dict={X: x_train_samples, y: y_train_samples})
    
# TODO
# Once in every 100 epochs save the best model with highest validation accuracy  
# saver.save(sess, os.path.join('Data', 'tf_models','model.ckpt'))


In [ ]:
# TODO
# Use higher level API for feedforward insted of explicit matrix multiplication

In [ ]:
# TODO
# Add more layers and repeat the process